###  Desafio 🚀
---

Nessa etapa do desafio, será realizada a leitura de um arquivo oriundo de webscraping, o csv "concert_tours_by_women.csv" em que possui dados referentes às turnês de cantoras. Assim, será feito um tratamento de dados inicial para posteriormente analisar as informações contidas no mesmo. Seguirei uma sequência de ações nas quais estarei detalhando um pouco mais sobre os códigos que usei para execução dos passos.
   

Análise inicial
__________________________________________________________________________________________________
Inicialmente importei as bibliotecas Pandas, que será usada para carregar, manipular e tratar os dados, e Re, que oferece suporte a expressões regulares para limpeza textual avançada. Para ler o arquivo e armazená-lo no DataFrame (df), utilizei a biblioteca Pandas e fiz a análise inicial dos dados.

In [ ]:
import pandas as pd

df = pd.read_csv("concert_tours_by_women.csv") 

- Visualizando melhor as primeiras linhas para entender um pouco mais sobre as colunas e como os dados estão organizados:

In [2]:
df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjustedgross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023-2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008-2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018-2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]


- Conhecendo as dimensões do DataFrame:

In [3]:
df.shape

(20, 11)

- Visualizando os valores vazios:

In [5]:
df.isnull().sum()

Rank                                0
Peak                               11
All Time Peak                      14
Actual gross                        0
Adjustedgross (in 2022 dollars)     0
Artist                              0
Tour title                          0
Year(s)                             0
Shows                               0
Average gross                       0
Ref.                                0
dtype: int64

- Informações da base (tipos de dados, linhas não nulas, número de entradas):

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Rank                             20 non-null     int64 
 1   Peak                             9 non-null      object
 2   All Time Peak                    6 non-null      object
 3   Actual gross                     20 non-null     object
 4   Adjustedgross (in 2022 dollars)  20 non-null     object
 5   Artist                           20 non-null     object
 6   Tour title                       20 non-null     object
 7   Year(s)                          20 non-null     object
 8   Shows                            20 non-null     int64 
 9   Average gross                    20 non-null     object
 10  Ref.                             20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


Tratamento de Dados
__________________________________________________________________________________________________
Após a análise inicial dos dados e compreendendo um pouco mais sobre a nossa base seguirei com o tratamento de dados. Inicialmente, removi as colunas que não seriam utilizadas no projeto, como Peak, All Time Peak e Ref, utilizando o método .drop(), passando o parâmetro columns com os nomes das colunas. Em seguida, renomeei as colunas restantes com nomes padronizados, usando o atributo .columns com uma nova lista de nomes.

In [6]:
df = df.drop(columns=["Peak", "All Time Peak", "Ref."])

df.columns = [
    "Rank",
    "Actual gross",
    "Adjusted gross (in 2022 dollars)",
    "Artist",
    "Tour title",
    "Year(s)",
    "Shows",
    "Average gross"
]

Na etapa de limpeza dos valores monetários, selecionei as colunas "Actual gross", "Adjusted gross (in 2022 dollars)" e "Average gross", pois elas continham símbolos como cifrão ($), vírgula (,) e referências entre colchetes ([1], [b]). Usei .replace() com expressões regulares para remover esses caracteres e, em seguida, converti os valores para o tipo float com .astype(float), garantindo que os dados estivessem em formato numérico.

In [7]:
monetary_columns = ["Actual gross", "Adjusted gross (in 2022 dollars)", "Average gross"]
for col in monetary_columns:
    df[col] = df[col].replace(r'\[.*?\]', '', regex=True)  
    df[col] = df[col].replace(r'[\$,]', '', regex=True)      
    df[col] = df[col].astype(float)   

Depois disso, iniciei a limpeza das colunas de texto "Artist" e "Tour title". Utilizei expressões regulares novamente para remover qualquer conteúdo entre colchetes ([...]), símbolos especiais como †, ‡, §, pontuações como *, :, ., ?, !, ,, e também substituí o caractere & por "and", com o objetivo de padronizar os nomes e facilitar análises posteriores. Também adicionei comandos para remover espaços duplicados com .str.replace(r'\s{2,}', ' ', regex=True) e espaços desnecessários nas bordas com .str.strip().

In [8]:
text_columns = ["Artist", "Tour title"]
for col in text_columns:
    df[col] = df[col].replace(r'\[.*?\]', '', regex=True)            
    df[col] = df[col].replace(r'[†‡§]', '', regex=True)             
    df[col] = df[col].replace(r'[*:.\?!,]', '', regex=True)         
    df[col] = df[col].str.replace('&', 'and')                        
    df[col] = df[col].str.replace(r'\s{2,}', ' ', regex=True)        
    df[col] = df[col].str.strip()      

A seguir, tratei a coluna "Year(s)", que contém intervalos de anos no formato "2008-2009" ou apenas um ano como "2023". Usei .str.split('-', expand=True) para separar em duas colunas: "Start year" e "End year". Quando não havia ano final, preenchi a coluna "End year" com o mesmo valor do início usando .fillna().

In [9]:
df[["Start year", "End year"]] = df["Year(s)"].str.split('-', expand=True)
df["End year"] = df["End year"].fillna(df["Start year"])  
df = df.drop(columns=["Year(s)"])

Em seguida, reorganizei a ordem das colunas do DataFrame para seguir o padrão exigido no desafio, usando uma nova lista com a sequência desejada. 

In [11]:
df = df[[
    "Rank", "Actual gross", "Adjusted gross (in 2022 dollars)",
    "Artist", "Tour title", "Shows", "Average gross", "Start year", "End year"
]]

Por fim, exportei o DataFrame já limpo e formatado para um novo arquivo chamado csv_limpo.csv, utilizando a função .to_csv() com index=False para evitar que o índice numérico do pandas fosse salvo no arquivo.

In [ ]:
df.to_csv("csv_limpo.csv", index=False)